In [44]:
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm
import warnings
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import json

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
pd.set_option('display.max_columns', None)

In [54]:
df = pd.read_csv('dirty_main2.csv')
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
print(list(df.columns))

['key', '%c1m', '%c1m_lag1', '%c1m_lag2', '%c1m_lag3', '%c2m', '%c2m_lag1', '%c2m_lag2', '%c2m_lag3', '%c3m', '%c3m_lag1', '%c3m_lag2', '%c3m_lag3', '%c4m', '%c4m_lag1', '%c4m_lag2', '%c4m_lag3', '%c5m', '%c5m_lag1', '%c5m_lag2', '%c5m_lag3', '%c10m', '%c10m_lag1', '%c10m_lag2', '%c10m_lag3', '%c15m', '%c15m_lag1', '%c15m_lag2', '%c15m_lag3', '%c20m', '%c20m_lag1', '%c20m_lag2', '%c20m_lag3', '%c25m', '%c25m_lag1', '%c25m_lag2', '%c25m_lag3', '%c30m', '%c30m_lag1', '%c30m_lag2', '%c30m_lag3', '%c45m', '%c45m_lag1', '%c45m_lag2', '%c45m_lag3', '%c60m', '%c60m_lag1', '%c60m_lag2', '%c60m_lag3', '%c75m', '%c75m_lag1', '%c75m_lag2', '%c75m_lag3', '%c90m', '%c90m_lag1', '%c90m_lag2', '%c90m_lag3', '%c105m', '%c105m_lag1', '%c105m_lag2', '%c120m', '%c120m_lag1', '%c120m_lag2', '%c135m', '%c135m_lag1', '%c150m', '%c150m_lag1', '%c165m', '%c165m_lag1', '%c180m', '%c180m_lag1', '%c195m', '%c210m', '%c225m', '%c240m', '%c255m', '%c270m', '%c285m', '%c300m', '%c315m', '%c330m', '%c345m', '%c360m'

<h1>Categorical</h1>

In [55]:
for col in ['key','ticker','date','market_cap','type']:
    print(df[col].describe())
    print(df[col].isna().sum())
    print('--------------------')

count            232666
unique           232666
top       AA_2021-03-18
freq                  1
Name: key, dtype: object
0
--------------------
count     232666
unique      5997
top         MIGI
freq         284
Name: ticker, dtype: object
0
--------------------
count         232666
unique           935
top       2022-01-26
freq            1411
Name: date, dtype: object
0
--------------------
count    2.269850e+05
mean     5.869796e+08
std      1.198726e+10
min      7.040000e+00
25%      2.041351e+07
50%      6.881465e+07
75%      2.412486e+08
max      3.469584e+12
Name: market_cap, dtype: float64
5681
--------------------
count     223565
unique         9
top           CS
freq      206179
Name: type, dtype: object
9101
--------------------


In [56]:
#fill any missing type data
for idx,row in df[df['type'].isna()].iterrows():
    id = df[df['ticker'] == row['ticker']]['type'].first_valid_index()
    if id != None:
        new_type = df.at[id, 'type']
        df.at[idx, 'type'] = new_type
filled = df['type'].isna().sum()
print(f'Filled {filled} type columns nans. {9101 - filled} remain.')

Filled 349 type columns nans. 8752 remain.


In [57]:
#drop wrong types
length = len(df)
df = df[df['type'].isin(['CS', 'OS', 'ADRC']) | df['type'].isna()]
print(f'{length - len(df)} obsv dropped: wrong type somehow got through')

413 obsv dropped: wrong type somehow got through


In [58]:
#remove unreliable market cap estimates
length = len(df[df['market_cap'] < 50000])
for idx,row in df[df['market_cap'] < 50000].iterrows():
    df.at[idx,'market_cap'] = np.nan
print(f'{length} mkt cap obsv altered: likely incorrect market cap')

501 mkt cap obsv altered: likely incorrect market cap


<h1>Previous % Returns</h1>

In [95]:
#Drop wrong values
length = len(df)
df = df[df['%cpah'] < 7]
df = df[df['%c1d'] < 12]
df = df[df['%c2d'] < 20]
df = df[df['%c3d'] < 28]
print(f'{length - len(df)} obsv dropped: incorrect values')

0 obsv dropped: incorrect values


In [97]:
pct_feats = [col for col in df.columns if '%' in col and 'lag' not in col and 'screen' not in col]
lag_feats = [col for col in df.columns if 'lag' in col]
for col in pct_feats + lag_feats:
    print(df[col].describe())
    print('--------------------')

count    229705.000000
mean         -0.002011
std           0.006848
min          -0.474047
25%          -0.003889
50%          -0.000553
75%           0.000260
max           0.223890
Name: %c1m, dtype: float64
--------------------
count    229705.000000
mean         -0.003715
std           0.010954
min          -0.279720
25%          -0.007726
50%          -0.001582
75%           0.000495
max           0.308611
Name: %c2m, dtype: float64
--------------------
count    203878.000000
mean         -0.004807
std           0.013390
min          -0.327676
25%          -0.010344
50%          -0.002737
75%           0.000822
max           0.385293
Name: %c3m, dtype: float64
--------------------
count    211563.000000
mean         -0.005635
std           0.014875
min          -0.354640
25%          -0.011915
50%          -0.003663
75%           0.000681
max           0.565684
Name: %c4m, dtype: float64
--------------------
count    206437.000000
mean         -0.006086
std           0.016284
min

<h1>Remove inaccurate return values due to low prices</h1>

In [102]:
returns = ['vwap_return', 'vwap_ft_return', 'ask_bid_return', 'mid_return','vwap_sfill_return']
for col in returns:
    print(df[col].describe())
    print('--------------------')

count    161725.000000
mean          0.007926
std           0.285716
min          -0.901396
25%          -0.015802
50%           0.002883
75%           0.022325
max          62.988048
Name: vwap_return, dtype: float64
--------------------
count    210373.000000
mean          0.008564
std           0.248366
min          -0.905215
25%          -0.013392
50%           0.003854
75%           0.022998
max          62.944223
Name: vwap_ft_return, dtype: float64
--------------------
count    229497.000000
mean         -0.018817
std           0.232323
min          -0.946346
25%          -0.042879
50%          -0.018289
75%           0.000000
max          61.235067
Name: ask_bid_return, dtype: float64
--------------------
count    229497.000000
mean          0.012370
std           0.236836
min          -0.944229
25%          -0.009662
50%           0.008377
75%           0.027094
max          62.431373
Name: mid_return, dtype: float64
--------------------
count    229548.000000
mean         -0.

In [100]:
#remove inaccurate returns due to 0 spread values
df.loc[df[['close_ask','open_bid']].eq(0).any(axis=1), ['ask_bid_return','mid_return','vwap_sfill_return']] = np.nan

In [106]:
#remove inaccurate returns
df = df[df['vwap_return'] < 7]

In [107]:
df['vwap_return'].describe()

count    161711.000000
mean          0.005857
std           0.088421
min          -0.901396
25%          -0.015804
50%           0.002880
75%           0.022318
max           5.911149
Name: vwap_return, dtype: float64

<h1>Organize</h1>

In [110]:
pct_feats = [col for col in df.columns if '%' in col and 'lag' not in col and 'screen' not in col]
lag_feats = [col for col in df.columns if 'lag' in col]
screen = [col for col in df.columns if 'screen' in col]
dollar_volumes = [col for col in df.columns if 'dv' in col]
returns = ['vwap_return', 'vwap_ft_return', 'ask_bid_return', 'mid_return','vwap_sfill_return']
entry_desc = ['first_tick_time','close_quote_time','open_quote_time']
indexes = [col for col in df.columns if '_pc' in col or '_ah' in col]
org = df[['key','ticker','date','market_cap','type'] + screen + pct_feats + lag_feats + dollar_volumes + entry_desc + returns + indexes]
org

,key,ticker,date,market_cap,type,%c1m_screen,%c2m_screen,%c3m_screen,%c4m_screen,%c5m_screen,%c10m_screen,%c15m_screen,%c20m_screen,%c25m_screen,%c30m_screen,%c45m_screen,%c60m_screen,%c75m_screen,%c90m_screen,%c105m_screen,%c120m_screen,%c135m_screen,%c150m_screen,%c165m_screen,%c180m_screen,%c195m_screen,%c210m_screen,%c225m_screen,%c240m_screen,%c255m_screen,%c270m_screen,%c285m_screen,%c300m_screen,%c315m_screen,%c330m_screen,%c345m_screen,%c360m_screen,%c375m_screen,%c1m,%c2m,%c3m,%c4m,%c5m,%c10m,%c15m,%c20m,%c25m,%c30m,%c45m,%c60m,%c75m,%c90m,%c105m,%c120m,%c135m,%c150m,%c165m,%c180m,%c195m,%c210m,%c225m,%c240m,%c255m,%c270m,%c285m,%c300m,%c315m,%c330m,%c345m,%c360m,%c375m,%cintra,%c1d,%cpah,%c2d,%c3d,%c1m_lag1,%c1m_lag2,%c1m_lag3,%c2m_lag1,%c2m_lag2,%c2m_lag3,%c3m_lag1,%c3m_lag2,%c3m_lag3,%c4m_lag1,%c4m_lag2,%c4m_lag3,%c5m_lag1,%c5m_lag2,%c5m_lag3,%c10m_lag1,%c10m_lag2,%c10m_lag3,%c15m_lag1,%c15m_lag2,%c15m_lag3,%c20m_lag1,%c20m_lag2,%c20m_lag3,%c25m_lag1,%c25m_lag2,%c25m_lag3,%c30m_lag1,%c30m_lag2,%c30m_lag3,%c45m_lag1,%c45m_lag2,%c45m_lag3,%c60m_lag1,%c60m_lag2,%c60m_lag3,%c75m_lag1,%c75m_lag2,%c75m_lag3,%c90m_lag1,%c90m_lag2,%c90m_lag3,%c105m_lag1,%c105m_lag2,%c120m_lag1,%c120m_lag2,%c135m_lag1,%c150m_lag1,%c165m_lag1,%c180m_lag1,dv1,dv2,dv3,dv4,dv5,dv10,dv15,dv20,dv25,dv30,dv45,dv60,dv75,dv90,dv105,dv120,dv135,dv150,dv165,dv180,dv195,dv210,dv225,dv240,dv255,dv270,dv285,dv300,dv315,dv330,dv345,dv360,dv375,dv389,15:59_dv,9:30_dv,first_tick_time,close_quote_time,open_quote_time,vwap_return,vwap_ft_return,ask_bid_return,mid_return,vwap_sfill_return,SPY_ah,SPY_pc_day,SPY_pc_timed,VONE_ah,VONE_pc_day,VONE_pc_timed,IWM_ah,IWM_pc_day,IWM_pc_timed,VTHR_ah,VTHR_pc_day,VTHR_pc_timed,VTI_ah,VTI_pc_day,VTI_pc_timed
0,AA_2021-03-18,AA,2021-03-18,5.835747e+09,CS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0.001304,0.000257,-0.000240,0.000876,0.002279,0.005028,-0.001882,-0.005692,-0.009080,-0.017786,-0.027054,-0.034564,-0.026753,-0.036191,-0.043400,-0.054587,-0.052441,-0.049500,-0.048593,-0.048108,-0.064327,-0.062552,-0.063309,-0.064734,-0.062746,-0.056270,-0.057924,-0.053594,-0.051319,-0.047369,-0.028878,-0.025162,-0.014105,-0.015161,-0.019805,-0.004715,0.056233,0.004510,-0.001046,-0.000497,0.001117,0.000620,0.001675,0.001843,0.002794,0.002902,-0.002772,0.003521,-0.001717,-0.006073,0.002743,-0.006876,-0.003817,-0.010667,-0.012163,-0.004185,-0.015933,-0.009436,-0.007719,-0.016297,-0.012952,0.000353,-0.019314,0.001508,-0.011207,-0.017082,-0.001685,-0.019087,-0.009391,-0.016860,0.004573,-0.020740,0.006854,-0.017467,-0.023372,-0.014528,0.010372,-0.012364,-0.008575,0.032963,-0.020021,0.011982,-0.010732,0.042311,-0.004041,-0.004307,0.001286,0.024105,1.752712e+06,3.233943e+06,4.604353e+06,7.170900e+06,8.195136e+06,1.548011e+07,2.021786e+07,2.379555e+07,2.802238e+07,3.098273e+07,3.794163e+07,4.189486e+07,4.685407e+07,5.312496e+07,5.786741e+07,6.280956e+07,6.727368e+07,7.079690e+07,7.377625e+07,8.085689e+07,8.774379e+07,9.386434e+07,1.004121e+08,1.044643e+08,1.109343e+08,1.187554e+08,1.286101e+08,1.374654e+08,1.444339e+08,1.622406e+08,1.710017e+08,1.773235e+08,1.848873e+08,2.007916e+08,3.327471e+06,2.294951e+07,1.0,15:59,09:30,-0.000288,-0.002512,-0.002566,0.000481,-0.000288,-0.004087,-0.014550,-0.014223,0.000327,-0.015562,-0.016045,-0.002975,-0.030433,-0.030132,0.002294,-0.016897,-0.016897,0.000683,-0.018199,-0.017241
1,AA_2021-12-01,AA,2021-12-01,8.191376e+09,CS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.001808,0.001455,-0.000678,-0.001797,-0.003999,-0.008682,-0.014018,-0.007383,-0.001923,-0.000805,-0.002443,-0.011354,-0.007228,-0.014910,-0.014236,-0.020456,-0.016145,-0.022108,-0.017257,-0.026909,-0.036974,-0.038382,-0.039899,-0.036755,-0.045391,-0.052047,-0.053458,-0.053220,-0.061309,-0.081481,-0.079679,-0.068134,-0.064853,-0.079975,-0.053084,0.029228,-0.103378,-0.081126,-0.000352,-0.002130,-0.001119,-0.003247,-0.003821,-0.000068,-0.004936,-0.002333,-0.002757,-0.003888,-0.005016,-0.002951,-

In [111]:
org.to_csv('main.csv')